In [4]:
import numpy as np
import random
import time
import pandas as pd
import operator
from collections import defaultdict
np.set_printoptions(suppress=True)

In [112]:
#[int(i) for i in split_line[3:]]
# Loading the single planar graph of 500 vertices
data = defaultdict(list)
for line in open("Graph500.txt"):
    split_line=line.split()
    ID_vertex = split_line[0]
    num_connected_vertices  = split_line[2]
    ID_connected_vertices = split_line[3:]
    if (ID_vertex) not in data.keys():
        data[ID_vertex].append(int(num_connected_vertices))
        data[ID_vertex].append(ID_connected_vertices)
df = pd.DataFrame(data.values(),columns = ['Number of connected vertices','ID connected vertices'])

In [119]:
df.iloc[0][1]

['28', '102', '162', '233', '360', '393', '460', '500']

In [126]:
df.index[1]

1